In [1]:
import numpy as np 
import os
from glob import glob
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import keras
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D,Dense, Flatten
from keras.models import model_from_json
import datetime

from keras.preprocessing.image import ImageDataGenerator

C:\Progra_Files\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
DATA_PATH = "./Dataset/Custom/"
img_dims = (120, 320)

lookup = dict()
reverselookup = dict()
count = 0

for j in os.listdir(DATA_PATH+'TRAIN/'):
    if not j.startswith('.'):
        lookup[j] = count	
        reverselookup[count] = j
        count = count + 1
print(reverselookup)

{0: 'fist', 1: 'four', 2: 'l', 3: 'ok', 4: 'palm', 5: 'three', 6: 'two'}


In [14]:
train_datagen = ImageDataGenerator(
                                   width_shift_range = 0.10,
                                   height_shift_range = 0.10,
                                   shear_range = 0.3,
                                   rescale=1./255,
                                   validation_split=0.2
                                   )

test_datagen = ImageDataGenerator(rescale=1./255)

train_data_generator = train_datagen.flow_from_directory(
                                                         directory = DATA_PATH + "Train/",
                                                         target_size = img_dims,
                                                         batch_size = 64,
                                                         class_mode = 'categorical',
                                                         subset = 'training',
                                                         color_mode = "grayscale",
                                                        )

validation_data_generator = train_datagen.flow_from_directory( 
                                                                   directory = DATA_PATH + "Train/",
                                                                   target_size = img_dims,
                                                                   batch_size = 64,
                                                                   class_mode = 'categorical',
                                                                   color_mode = "grayscale",
                                                                   subset = 'validation'
                                                                  )
test_data_generator = test_datagen.flow_from_directory(
                                                          directory = DATA_PATH + "Test/",
                                                          target_size = img_dims,
                                                          batch_size = 64,
                                                          class_mode = 'categorical',
                                                          color_mode = "grayscale",
                                                          shuffle = False
                                                      )

Found 313 images belonging to 7 classes.
Found 74 images belonging to 7 classes.
Found 74 images belonging to 7 classes.


In [15]:
model = Sequential()
model.add(Conv2D(32, (5, 5), strides=(2, 2), activation='relu', input_shape=(120, 320,1))) 
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu')) 
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(7, activation='softmax'))

model.compile(optimizer = 'rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

print(model.summary())



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_10 (Conv2D)           (None, 58, 158, 32)       832       
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 29, 79, 32)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 27, 77, 64)        18496     
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 13, 38, 64)        0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 11, 36, 64)        36928     
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 5, 18, 64)         0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 5760)              0         
__________

In [16]:
###############################################
EPOCHS = 10
BATCH_SIZE = 64
#############################################
t0 = datetime.datetime.now()
history = model.fit_generator(
                               train_data_generator,
                               steps_per_epoch = train_data_generator.samples/BATCH_SIZE,
                               epochs = EPOCHS,
                               validation_data = validation_data_generator,
                               validation_steps = validation_data_generator.samples/BATCH_SIZE,
                               shuffle = True
                             )
training_time = datetime.datetime.now() - t0
print("Total training time =", training_time)

Epoch 1/10
5/4 [==============================] - 5s 1s/step - loss: 2.0674 - acc: 0.2784 - val_loss: 1.7664 - val_acc: 0.4730
Epoch 2/10
5/4 [==============================] - 6s 1s/step - loss: 1.6765 - acc: 0.4117 - val_loss: 1.3840 - val_acc: 0.4595
Epoch 3/10
5/4 [==============================] - 6s 1s/step - loss: 1.2875 - acc: 0.5410 - val_loss: 1.4734 - val_acc: 0.4324
Epoch 4/10
5/4 [==============================] - 7s 1s/step - loss: 0.9721 - acc: 0.6884 - val_loss: 1.1220 - val_acc: 0.6216
Epoch 5/10
5/4 [==============================] - 6s 1s/step - loss: 0.8486 - acc: 0.6934 - val_loss: 0.6414 - val_acc: 0.7838
Epoch 6/10
5/4 [==============================] - 7s 1s/step - loss: 0.6292 - acc: 0.7863 - val_loss: 0.6965 - val_acc: 0.6892
Epoch 7/10
5/4 [==============================] - 7s 1s/step - loss: 0.5618 - acc: 0.8055 - val_loss: 0.4426 - val_acc: 0.8243
Epoch 8/10
5/4 [==============================] - 7s 1s/step - loss: 0.5615 - acc: 0.8356 - val_loss: 0.5052 - 

In [17]:
test_data_generator.reset()

actual_class_indices = test_data_generator.classes
preds = model.predict_generator(test_data_generator, steps = test_data_generator.samples/BATCH_SIZE)
predicted_class_indices = np.argmax(preds, axis=1)

right_predictions = 0

for actual, predicted in zip(actual_class_indices, predicted_class_indices):
    if actual == predicted: 
        right_predictions += 1

print("Accuracy =", right_predictions/len(actual_class_indices)*100)

Accuracy = 79.72972972972973


In [18]:
from keras.models import model_from_json

model_json = model.to_json()
with open("./Models/GestureDetectionModel_2.json", "w") as json_file:
    json_file.write(model_json)
 # serialize weights to HDF5
model.save_weights("./Models/GestureDetectionModel_2_weights.h5")
print("Saved model to disk")

Saved model to disk
